I believe 1000000 data could fit the 8/2 rule, it means only  20% data need to be optimized. So I try to narrow down these data.

In [ ]:
import os
os.system("date")


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input"))

In [ ]:
#input file 
submission=pd.read_csv('../input/sample_submission_random_v2.csv')
child=pd.read_csv('../input/child_wishlist_v2.csv',header=None)
gift=pd.read_csv('../input/gift_goodkids_v2.csv',header=None)

GIFT LIST WORK

In [ ]:
#I decide start from gift list
#first I build a blanket table
tempSub=pd.DataFrame()
tempSub['GiftId']=0
tempSub['ChildId']=0
tempSub['Tier']=0
tempSub

In [ ]:
#this is gift list looks like
gift.head()

In [ ]:
#looks Santa gift plant has a list, so let me try use their list and see what will happen
#now I fill table by gift wish list
for i in range(1,1001):
    temp=gift.loc[:,[0,i]]
    temp.columns=['GiftId','ChildId']
    temp['Tier']=i
    tempSub=tempSub.append(temp)
    print(u"\r"+str(i),end='')

In [ ]:
#I got a list, this list tell which child should give which gift and mark with tier from gift list
#looks good to me
print(tempSub.shape)
tempSub.head()

In [ ]:
#but there is a problem
#gift list only cover half of child, that because some are on multi gift list
len(tempSub['ChildId'].value_counts())

In [ ]:
#now I have to find the best option
#filter low tier option
tempGroup=tempSub['Tier'].groupby(tempSub['ChildId'])
tempGroup=tempGroup.min().to_frame()

In [ ]:
#find the best tier for child
tempGroup.head()

In [ ]:
#I do know how to convert it to normal talbe so I use another way to finish it.
tempGroup.to_csv('tempGroup.csv')
tempGroup=pd.read_csv('tempGroup.csv')
tempGroup.head()

In [ ]:
#merge to produce pure result with top tier option
resultGiftSide = pd.merge(tempGroup,tempSub , how='inner', on=['ChildId', 'Tier'])
print(len(resultGiftSide))
resultGiftSide.head()
#the number is a litter bigger because some child have mult gift sent wish list with same tier

In [ ]:
#locate these child ID
tempCounts=resultGiftSide['ChildId'].value_counts()
tempCounts=tempCounts.to_frame()
tempCounts=tempCounts[tempCounts['ChildId']>1]
tempCounts=tempCounts.index
print(len(tempCounts))
temp=resultGiftSide[resultGiftSide['ChildId'].isin(tempCounts)]
temp=temp.sort_values(by='Tier')
temp.head()
#they are small group and variable, I decide leave behind, even they are so popular in the gift with list.

In [ ]:
#now I produce something about gift side result
resultGiftSideConstant=resultGiftSide[~resultGiftSide['ChildId'].isin(tempCounts)]
resultGiftSideVarable=resultGiftSide[resultGiftSide['ChildId'].isin(tempCounts)]

In [ ]:
print(resultGiftSideConstant.shape)
print(resultGiftSideVarable.shape)
print(len(resultGiftSideConstant))
resultGiftSideVarable.head()

In [ ]:
#save it (maybe not necessary)
#resultGiftSideConstant.to_csv('resultGiftSideConstant.csv')
#resultGiftSideVarable.to_csv('resultGiftSideVarable.csv')

In [ ]:
#after I narrow down half of gift wish list, now I have to check stock situation.
stock=resultGiftSideConstant['GiftId'].value_counts()
stock=stock.to_frame()
stock.columns=['Used']
stock.head()
stock['left']=1000-stock['Used']
stock.head()
#from now on gift side work should close

CHILD LIST WORK

In [ ]:
#previous gift side work told me only half child are cover on the gift list.
#That means half of child a free will to find their best wish without conflict with gift wish list, good for them.
childFree=child[~child[0].isin(resultGiftSideConstant['ChildId'].values)]
childFree=childFree[~childFree[0].isin(resultGiftSideVarable['ChildId'].values)]

In [ ]:
#these child are uncover on the gift wish list. So they are free will finding the best wish they want as long as stock could make it.
print(len(childFree))
childFree.head()

In [ ]:
#check stock
stock.head()

In [ ]:
#find out these child first tier wish.
childFreeTier1Gift=childFree[1].value_counts().to_frame()
childFreeTier1Gift.columns=['order']
childFreeTier1Gift.head()

In [ ]:
#compare with stock
matchStock=pd.merge(stock,childFreeTier1Gift,left_on=stock.index.values,right_on=childFreeTier1Gift.index.values,how='left')

In [ ]:
matchStock.columns=['GiftId','Used','left','order']
matchStock.head()

In [ ]:
matchStock['backup']=matchStock['left']-matchStock['order']
matchStock.head()
#here we find freewill child have some wish make stock can not fulfill.

In [ ]:
#we filter these stock which can fulfill list
freeWillWithoutCompititionGift=matchStock[matchStock['backup']>=0]
freeWillWithoutCompititionGift.head()

In [ ]:
#these child wish can fulfill by stock without question
freeChildConstant=childFree[childFree[1].isin(freeWillWithoutCompititionGift['GiftId'])]
freeChildConstant.head()

In [ ]:
#let's produce the list
freeChildConstant=freeChildConstant.loc[:,[0,1]]
freeChildConstant.columns=['ChildId','GiftId']
#freeChildConstant.to_csv('freeChildConstant.csv')
print(len(freeChildConstant))
freeChildConstant.head()

COMBINE WHAT WE FIND OUT

In [ ]:
#after previous work we try to find a list that should be constant and directly submit without question.
#here we organize what we find.
print(len(resultGiftSideConstant))
resultGiftSideConstant.head()

In [ ]:
#let's filter something could varable
#remove tail that could in the compitition field with both gift list and child list because the evaluation things.
directSubGiftSide=resultGiftSideConstant[resultGiftSideConstant['Tier']<900]
#remove twins
directSubGiftSide=directSubGiftSide[directSubGiftSide['ChildId']>45000]
#what we left is real gift side constant list should submit without question

In [ ]:
directSubGiftSide=directSubGiftSide.loc[:,['ChildId','GiftId']]
print(len(directSubGiftSide))
directSubGiftSide.head()

In [ ]:
directSubChildSide=freeChildConstant[freeChildConstant['ChildId']>45000]
print(len(directSubChildSide))
directSubChildSide.head()

In [ ]:
#now let's combine both constant list finish the constant submit list.
directSubConstant=directSubChildSide.append(directSubGiftSide,ignore_index=True)
#directSubConstant.to_csv('directSubConstant.csv')
print(len(directSubConstant))
directSubConstant.head()

Now whole data we fixed more than half, only 40% data left.
I have a question here, we narrow it down more or just end here.
If we end fixed work here, there are 400000 data left we could use some kind of optimizer tech to find a solution(maybe the Genetic Algorithm tech).
but 400000 for GA could still take sometime.(kaggle has time limit for user's kernel)
otherside  400000 left, means there is a big field for GA to work. So stop narrow down it might be good choice.

In [ ]:
gift.head()

In [ ]:
#before we continue, let's check our assert
#first we shrink the gift list casue the 0-990 is fixed, so what left is a 10 wide list, just like child wish list.
#gift side
templist=list(range(901,1001))
templist.insert(0,0)
giftTail=gift.loc[:,templist]
#here we convert tier to score
templist=list(range(100,0,-1))
templist.insert(0,'GiftId')
giftTail.columns=[templist]
print(len(giftTail))
giftTail.head()

In [ ]:
#child side
childTail=child[~child[0].isin(directSubConstant['ChildId'])]
#here we convert tier to score
templist=list(range(100,0,-1))
templist.insert(0,'ChildId')
childTail.columns=[templist]
print(len(childTail))
childTail.head()

In [ ]:
#stock check
#stock used
stockUsed=directSubConstant['GiftId'].value_counts().to_frame()
stockUsed.columns=['used']
stockUsed['left']=1000-stockUsed['used']
stockCheck=stockUsed
print(len(stockCheck))
print(stockCheck['left'].sum())
stockCheck.head()

CHECK POINT

In [ ]:
#All we work next is base on these three file. Previous thing looks good (may be not best) but we are going to move on.
stock1=stockCheck
gift1=giftTail
child1=childTail

from here you can use your optimer to find the best solution when most of data have been fixed. 
And I believe these fixed data only help you improve efficient of your optimizer and only cost a few opportunity cost. 
cheer up!